# Подгонка ARIMA (пакет [`sktime`](https://www.sktime.net/en/stable/))

In [1]:
import numpy as np
import pandas as pd

from sktime.forecasting.arima import ARIMA, AutoARIMA
from sktime.utils.plotting import plot_series

import pandas_datareader.data as web

# настройки визуализация
import matplotlib.pyplot as plt

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)
# Не показывать ValueWarning, ConvergenceWarning из statsmodels
from statsmodels.tools.sm_exceptions import ValueWarning, ConvergenceWarning
warnings.simplefilter('ignore', category=ValueWarning)
warnings.simplefilter('ignore', category=ConvergenceWarning)

## Подгонка ARIMA заданного порядка без сноса

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) недельные данные по 3-Month Treasury Bill Secondary Market Rate (Symbol [`WTB3MS`](https://fred.stlouisfed.org/series/WTB3MS)) с 2000-01-01 по 2023-12-31 и создадим датафрейм `rate`

In [2]:
y = web.DataReader(name='WTB3MS', data_source='fred', start='2000-01-01', end='2023-12-31')

Подгоним модель ARMA(2,1,2) без сноса для `y`

Спецификация

$$
	(1-\phi_1L-\phi_2 L^2)(1-L) y_t=u_t+\theta_1 u_{t-1}+\theta_2 u_{t-2}
$$

In [3]:
forecaster = ARIMA(order=(2,1,2), trend='n')
forecaster.fit(y)
forecaster.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                 1252
Model:               SARIMAX(2, 1, 2)   Log Likelihood                1386.078
Date:                Mon, 18 Mar 2024   AIC                          -2762.157
Time:                        15:55:45   BIC                          -2736.498
Sample:                    01-07-2000   HQIC                         -2752.511
                         - 12-29-2023                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.8581      0.116      7.396      0.000       0.631       1.086
ar.L2          0.1188      0.110      1.082      0.279      -0.096       0.334
ma.L1         -0.6665      0.117     -5.687      0.000      -0.896      -0.437
ma.L2         -0.2458      0.099     -2.478      0.013      -0.440      -0.051
sigma2         0.0064   6.72e-05     94.904      0.000       0.006       0.007
===================================================================================
Ljung-Box (L1) (Q):                   0.00   Jarque-Bera (JB):             81356.64
Prob(Q):                              0.96   Prob(JB):                         0.00
Heteroskedasticity (H):               0.49   Skew:                            -2.23
Prob(H) (two-sided):                  0.00   Kurtosis:                        42.25
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [4]:
forecaster.get_fitted_params()

{'ar.L1': 0.8581486121792634,
 'ar.L2': 0.11880003924782097,
 'ma.L1': -0.666457623042958,
 'ma.L2': -0.24581090047515824,
 'sigma2': 0.006382110157672486,
 'aic': -2762.156688218109,
 'aicc': -2762.108495447025,
 'bic': -2736.498195665775,
 'hqic': -2752.5111939549283}

## Подгонка ARMA заданного порядка со сносом

Загрузим из БД [`FRED`](https://fred.stlouisfed.org/) квартальные данные по ВВП (Symbol [`GDP`](https://fred.stlouisfed.org/series/GDP)) с 1990 Q1 по 2023 Q4 и создадим датафрейм `y=log(GDP)`

In [6]:
y = np.log(web.DataReader(name='GDP', data_source='fred', start='1990-1', end='2023-12' ) )

Подгоним модель ARMA(1,1,1) со сносом для `y` 

Спецификация

$$
	(1-\phi L)(1-L) y_t=\alpha_0+u_t+\theta u_{t-1}
$$

In [8]:
forecaster = ARIMA(order=(1,1,1), trend='t')
forecaster.fit(y)
forecaster.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                      y   No. Observations:                  136
Model:               SARIMAX(1, 1, 1)   Log Likelihood                 387.875
Date:                Mon, 18 Mar 2024   AIC                           -767.751
Time:                        15:57:30   BIC                           -756.130
Sample:                    01-01-1990   HQIC                          -763.028
                         - 10-01-2023                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
drift       3.909e-05      0.000      0.310      0.756      -0.000       0.000
ar.L1          0.6610      1.083      0.610      0.542      -1.461       2.783
ma.L1         -0.6140      1.118     -0.549      0.583      -2.806       1.578
sigma2         0.0002      1e-05     18.226      0.000       0.000       0.000
===================================================================================
Ljung-Box (L1) (Q):                   0.03   Jarque-Bera (JB):              5290.79
Prob(Q):                              0.86   Prob(JB):                         0.00
Heteroskedasticity (H):               3.46   Skew:                            -2.38
Prob(H) (two-sided):                  0.00   Kurtosis:                        33.30
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

# Автоматический выбор порядка модели

Выбор порядка по информационным критериям. Выведем только порядок подогнанной модели

In [14]:
forecaster = AutoARIMA(start_p=1, start_q=1, max_p=5, max_q=5, information_criterion='aic', seasonal=False)
forecaster.fit(y)
forecaster.get_fitted_params() #['order']

{'intercept': 0.011554757113466504,
 'sigma2': 0.00015721389010940986,
 'order': (0, 1, 0),
 'seasonal_order': (0, 0, 0, 0),
 'aic': -795.1978113081782,
 'aicc': -795.1069022172691,
 'bic': -789.3872617513014,
 'hqic': -792.8365668621092}

In [15]:
forecaster = AutoARIMA(start_p=1, start_q=1, max_p=5, max_q=5, information_criterion='bic', seasonal=False)
forecaster.fit(y)
forecaster.get_fitted_params()['order']

(0, 1, 0)

In [16]:
forecaster = AutoARIMA(start_p=1, start_q=1, max_p=5, max_q=5, information_criterion='bic', seasonal=False)
forecaster.fit(y)
forecaster.get_fitted_params()['order']

(0, 1, 0)